In [22]:
from decouple import config

In [23]:
assert config("ALPHA_VANTAGE_API_KEY", default=None, cast=str) is not None

In [24]:
ALPHA_VANTAGE_API_KEY = config("ALPHA_VANTAGE_API_KEY", default=None, cast=str)

In [28]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key


api_key = ALPHA_VANTAGE_API_KEY

params = {
    "api_key": ALPHA_VANTAGE_API_KEY,
    "ticker": "AAPL",
    "function": "TIME_SERIES_INTRADAY"
}

url = 'https://www.alphavantage.co/query?function={function}&symbol={ticker}&interval=1min&apikey={api_key}'.format(
    **params
)
r = requests.get(url)
data = r.json()

In [34]:
dataset_key = [x for x in list(data.keys()) if not x.lower() == "meta data"][0]
dataset_key

'Time Series (1min)'

In [40]:
results = data[dataset_key] # dict
# for timestamp_str in results.keys():
#     print(timestamp_str, results.get(timestamp_str))

In [42]:
timestamp_str = list(results.keys())[0]
timestamp_str

'2024-11-05 19:59:00'

datetime.datetime(2024, 11, 6, 0, 59, tzinfo=<UTC>)

In [45]:
result_val = results.get(timestamp_str)
result_val

{'1. open': '223.0999',
 '2. high': '223.1500',
 '3. low': '222.9000',
 '4. close': '223.0000',
 '5. volume': '3003'}

In [59]:
import pytz
from datetime import datetime
from decimal import Decimal

def transform_alpha_vantage_result(timestamp_str, result):
    # unix_timestamp = result.get('t') / 1000.0
    # utc_timestamp = datetime.fromtimestamp(unix_timestamp, tz=pytz.timezone('UTC'))
    timestamp_format = '%Y-%m-%d %H:%M:%S' 
    eastern = pytz.timezone("US/Eastern")
    utc = pytz.utc
    timestamp = eastern.localize(datetime.strptime(timestamp_str,timestamp_format)).astimezone(utc)
    return {
        'open_price': Decimal(result['1. open']),
        'close_price': Decimal(result['4. close']),
        'high_price': Decimal(result['2. high']),
        'low_price': Decimal(result['3. low']),
        'number_of_trades': None,
        'volume': int(result['5. volume']),
        'volume_weighted_average': None,
        'time': timestamp,
    }

In [60]:
transform_alpha_vantage_result(timestamp_str, result_val)

{'open_price': Decimal('223.0999'),
 'close_price': Decimal('223.0000'),
 'high_price': Decimal('223.1500'),
 'low_price': Decimal('222.9000'),
 'number_of_trades': None,
 'volume': 3003,
 'volume_weighted_average': None,
 'time': datetime.datetime(2024, 11, 6, 0, 59, tzinfo=<UTC>)}

In [61]:
for timestamp_str in results.keys():
    new_data = transform_alpha_vantage_result(timestamp_str, results.get(timestamp_str))
    print(new_data)

{'open_price': Decimal('223.0999'), 'close_price': Decimal('223.0000'), 'high_price': Decimal('223.1500'), 'low_price': Decimal('222.9000'), 'number_of_trades': None, 'volume': 3003, 'volume_weighted_average': None, 'time': datetime.datetime(2024, 11, 6, 0, 59, tzinfo=<UTC>)}
{'open_price': Decimal('222.9700'), 'close_price': Decimal('223.1000'), 'high_price': Decimal('223.1000'), 'low_price': Decimal('222.9000'), 'number_of_trades': None, 'volume': 5142, 'volume_weighted_average': None, 'time': datetime.datetime(2024, 11, 6, 0, 58, tzinfo=<UTC>)}
{'open_price': Decimal('222.8600'), 'close_price': Decimal('223.0000'), 'high_price': Decimal('223.1000'), 'low_price': Decimal('222.8499'), 'number_of_trades': None, 'volume': 415, 'volume_weighted_average': None, 'time': datetime.datetime(2024, 11, 6, 0, 57, tzinfo=<UTC>)}
{'open_price': Decimal('222.8400'), 'close_price': Decimal('222.8500'), 'high_price': Decimal('222.8656'), 'low_price': Decimal('222.8000'), 'number_of_trades': None, 'vo